In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

### Import data

In [2]:
#load the full training dataset in variable called npz
npz = np.load("Data/Audiobooks_data_train.npz")

# we stored our inputs and targets in the dataset, now assigned to npz, with keywords inputs and target
# We therefore can extract the inputs and targets by indexing on the variable npz and set the data types
train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

#load the full validation dataset in variable called npz
npz = np.load("Data/Audiobooks_data_validation.npz")

# store validation inputs and targets
validation_inputs = npz['inputs'].astype(np.float)
validation_targets = npz['targets'].astype(np.int)

#load the full testing dataset in variable called npz
npz = np.load("Data/Audiobooks_data_test.npz")

# store testing inputs and targets
test_inputs = npz['inputs'].astype(np.float)
test_targets = npz['targets'].astype(np.int)

In [3]:
# Testing the balance of the target datase
train_converts = np.where(train_targets == 1)[0].shape[0]
train_nonconverts = np.where(train_targets == 0)[0].shape[0]
validation_converts = np.where(validation_targets == 1)[0].shape[0]
validation_nonconverts = np.where(validation_targets == 0)[0].shape[0]
test_converts = np.where(test_targets == 1)[0].shape[0]
test_nonconverts = np.where(test_targets == 0)[0].shape[0]

print(f' There are {train_nonconverts} non-converts and {train_converts} converts in the training dataset targets for a ratio of {train_nonconverts/train_converts}.')
print(f' There are {validation_nonconverts} non-converts and {validation_converts} converts in the validation dataset targets for a ratio of {validation_nonconverts/validation_converts} .')
print(f' There are {test_nonconverts} non-converts and {test_converts} converts in the testing dataset targets for a ratio of {test_nonconverts/test_converts}.')






 There are 1796 non-converts and 1783 converts in the training dataset targets for a ratio of 1.007291082445317.
 There are 215 non-converts and 232 converts in the validation dataset targets for a ratio of 0.9267241379310345 .
 There are 226 non-converts and 222 converts in the testing dataset targets for a ratio of 1.018018018018018.



### Model


In [4]:

from tensorflow.keras.backend import sigmoid
def swish_activation(x, beta = 1):
    return (x * sigmoid(beta * x))

In [5]:
# Taking model from MNIST project and changing some parameters and hyperparameters
input_size = 10
output_size = 2
hidden_layer_size = 15

model = tf.keras.Sequential([
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                    tf.keras.layers.Dense(output_size,activation = 'softmax')   
        ])

#Optimizer and loss function
model.compile(optimizer='adam',loss = 'sparse_categorical_crossentropy',metrics=['accuracy'])


batch_size = 100
max_epochs = 100

# This earlystopping function in tf.keras.callbacks will monitor validation loss and stop the trianing process
# the first time the validation loss starts increasing
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs,
            train_targets,
            batch_size=batch_size,
            epochs = max_epochs,
            callbacks = [early_stopping],
            validation_data = (validation_inputs,validation_targets),
            verbose=2)

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 1s - loss: 6.5670 - accuracy: 0.5652 - val_loss: 1.4188 - val_accuracy: 0.5705
Epoch 2/100
3579/3579 - 0s - loss: 0.8215 - accuracy: 0.6270 - val_loss: 0.5850 - val_accuracy: 0.6242
Epoch 3/100
3579/3579 - 0s - loss: 0.4965 - accuracy: 0.6924 - val_loss: 0.4394 - val_accuracy: 0.7740
Epoch 4/100
3579/3579 - 0s - loss: 0.4246 - accuracy: 0.7784 - val_loss: 0.4357 - val_accuracy: 0.7136
Epoch 5/100
3579/3579 - 0s - loss: 0.4207 - accuracy: 0.7734 - val_loss: 0.4247 - val_accuracy: 0.7651
Epoch 6/100
3579/3579 - 0s - loss: 0.4235 - accuracy: 0.7508 - val_loss: 0.4299 - val_accuracy: 0.7181
Epoch 7/100
3579/3579 - 0s - loss: 0.4112 - accuracy: 0.7661 - val_loss: 0.4588 - val_accuracy: 0.7494


### Testing the model 

In [6]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

448/1 [=================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [7]:
print(f'Test Loss: {test_loss}, Test Accuracy: {round(100*test_accuracy,2)}%')

Test Loss: 0.4263248379741396, Test Accuracy: 74.78%


In [8]:
customers = np.loadtxt('Data/original.csv',delimiter=',')
customers

array([[9.9400e+02, 1.6200e+03, 1.6200e+03, ..., 5.0000e+00, 9.2000e+01,
        0.0000e+00],
       [1.1430e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0590e+03, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 3.8800e+02,
        0.0000e+00],
       ...,
       [3.1134e+04, 2.1600e+03, 2.1600e+03, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [3.2832e+04, 1.6200e+03, 1.6200e+03, ..., 0.0000e+00, 9.0000e+01,
        0.0000e+00],
       [2.5100e+02, 1.6740e+03, 3.3480e+03, ..., 0.0000e+00, 0.0000e+00,
        1.0000e+00]])

In [9]:
predictions = []
targets = customers[:,-1]
for i in range(customers.shape[0]-10000):
    predictions.append(model.predict_classes([[[[customers[i,1:-1]]]]]))

In [24]:
pred_df = pd.DataFrame([predictions,targets]).T.iloc[:4000]
pred_df.columns = ["predictions","actuals"]
pred_df["Difference"] = pred_df["predictions"] + pred_df["actuals"]
pred_df

,predictions,actuals,Difference
0,[[[0]]],0,[[[0.0]]]
1,[[[0]]],0,[[[0.0]]]
2,[[[1]]],0,[[[1.0]]]
3,[[[0]]],0,[[[0.0]]]
4,[[[0]]],0,[[[0.0]]]
...,...,...,...
3995,[[[0]]],0,[[[0.0]]]
3996,[[[0]]],0,[[[0.0]]]
3997,[[[0]]],0,[[[0.0]]]
3998,[[[0]]],0,[[[0.0]]]


In [26]:
wrong = 0
for row in pred_df.Difference:
    if row == 1:
        wrong += 1

In [33]:
right_accuracy = (len(pred_df) - wrong) / len(pred_df)

In [40]:
f'the model predicts if a new customer, not seen in training, \
will be a repeat customer or not with {round(right_accuracy*100,2)} accuracy'

'the model predicts if a new customer, not seen in training, will be a repeat customer or not with 83.9 accuracy'